In [1]:
import sqlite3
import pandas as pd
import os

conn = sqlite3.connect('challenge.db', timeout=5.0)
cursor = conn.cursor()

In [2]:
# On first glance, it seemed as if there were a lot more variables in the variables_list file than were accounted for in the table.

In [3]:
cursor.execute("SELECT COUNT(*) FROM variable_list")
row_count = cursor.fetchone()[0]

cursor.execute("SELECT \"Variable Code\" FROM variable_list")
variable_codes = [row[0] for row in cursor.fetchall()]

In [4]:
cursor.execute("PRAGMA table_info(five_hundred_cities)")
columns_500c = cursor.fetchall()
cursor.execute("PRAGMA table_info(access)")
columns_access = cursor.fetchall()

total_column_count = len(columns_500c) + len(columns_access)

In [5]:
print(f"Variables represented by rows in variables_list: {row_count}")
print(f"Variables represented in the columns: {total_column_count}")

Variables represented by rows in variables_list: 281
Variables represented in the columns: 109


In [6]:
# This confirmed that there were a number of variables for which I was missing values.

# Some of these missing variables seemed relevant to the assignment, like the number of grocery stores per county,
# so I downloaded the complete dataset from the USDA.

# Specifically, I wanted the sheets: "STORES", "RESTAURANTS", "ASSISTANCE", "INSECURITY", "PRICES_TAXES", "LOCAL", "HEALTH", and "SOCIOECONOMIC".

In [7]:
csv_dir = 'usda_csv'

csv_files = [
    'stores.csv',
    'restaurants.csv',
    'assistance.csv',
    'insecurity.csv',
    'prices_taxes.csv',
    'local.csv',
    'health.csv',
    'socioeconomic.csv'
]


cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()


for file in csv_files:
    file_path = os.path.join(csv_dir, file)
    table_name = os.path.splitext(file)[0]

    df = pd.read_csv(file_path)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()
    
conn.close()

In [8]:
# Last, I want to have a column for FIPS codes in the 500 Cities database so that I can merge it with the USDA data.

In [9]:
conn = sqlite3.connect('challenge.db')
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(five_hundred_cities)")
columns = cursor.fetchall()

column_names = [column[1] for column in columns]

if 'FIPS' not in column_names:
    cursor.execute("ALTER TABLE five_hundred_cities ADD COLUMN FIPS INTEGER")

cursor.execute("""
    UPDATE five_hundred_cities 
    SET FIPS = CASE 
                WHEN StateAbbr IN ('AL', 'AK', 'AR', 'AZ', 'CA', 'CO', 'CT') THEN SUBSTR(TRACTFIPS, 1, 4)
                ELSE SUBSTR(TRACTFIPS, 1, 5)
                END
""")

conn.commit()

cursor.close()
conn.close()

In [10]:
# Becuase the 500 Cities database has a finer level of detail than the USDA data, I want
# to make weighted averages for each of the values in the table.

In [19]:
conn = sqlite3.connect('challenge.db')
cursor = conn.cursor()

exclude_columns = ['index', 'StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID', 'Population2010', 'Geolocation', 'FIPS']

cursor.execute("PRAGMA table_info(five_hundred_cities)")
columns = cursor.fetchall()


variable_columns = [col[1] for col in columns if col[1] not in exclude_columns and '95CI' not in col[1]]

weighted_avg_query = ", ".join([f"ROUND(SUM({col} * Population2010) / SUM(Population2010), 1) AS {col}_weighted_avg" for col in variable_columns])

cursor.execute('DROP TABLE IF EXISTS "five_hundred_cities_summary"')

query = f"""
    CREATE TABLE "five_hundred_cities_summary" AS
    SELECT FIPS, StateAbbr, PlaceName, SUM(Population2010) AS Total_Population2010, {weighted_avg_query}
    FROM five_hundred_cities
    GROUP BY FIPS
"""
cursor.execute(query)
conn.commit()

conn.close()

In [12]:
# Now, I have more variables from the USDA database and I have a means of comparison with 500 Cities.

In [13]:
# One last thing I am missing is overall population figures for each county. A lot of the USDA figures are represented as percentages
# of the total population, so I want to know the actual population numbers in case I need them. 

In [14]:
conn = sqlite3.connect('challenge.db')
cursor = conn.cursor()

df = pd.read_csv('co-est2019-alldata.csv', dtype={'COUNTY': str}, encoding='latin1')
columns = ', '.join([f"{col} TEXT" for col in df.columns])

create_table_query = f"""
CREATE TABLE IF NOT EXISTS census (
    {columns}
);
"""

cursor.execute(create_table_query)

for _, row in df.iterrows():
    placeholders = ', '.join('?' * len(row))
    insert_query = f"INSERT INTO census VALUES ({placeholders})"
    cursor.execute(insert_query, tuple(row))

conn.commit()
conn.close()

In [16]:
conn = sqlite3.connect('challenge.db')
cursor = conn.cursor()

cursor.execute("ALTER TABLE census ADD COLUMN FIPS TEXT;")

cursor.execute("""
    UPDATE census
    SET FIPS = STATE || COUNTY;
""")

conn.commit()
conn.close()

OperationalError: duplicate column name: FIPS